# Análisis de imágenes en frecuencia

### Contenidos

- Transformada de Fourier bidimensional
- Espectro de imágenes sintéticas y naturales


In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from ipywidgets import interact, IntSlider

def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114])

## Transformada de Fourier bidimensional

La DFT se puede aplicar a funciones multi-dimensionales

En el caso discreto de una señal bidimensional $g[n_1, n_2]$ con índices $n_1 \in [0, N_1-1]$ y $n_2 \in [0, N_2-1]$ tenemos

$$
G[k_1, k_2] = \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} g[n_1, n_2] \exp \left ( -j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right)
$$
y su inversa

$$
g[n_1, n_2] = \frac{1}{N_1 N_2}\sum_{k_1=0}^{N_1-1} \sum_{k_2=0}^{N_2-1} G[k_1, k_2] \exp \left ( j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right)
$$

Notemos que

$$
\begin{align}
G[k_1, k_2] &= \sum_{n_1=0}^{N_1-1} \left(\sum_{n_2=0}^{N_2-1} g[n_1, n_2] \exp \left (-j2\pi \frac{n_2 k_2}{N_2}\right) \right) \exp \left (-j2\pi \frac{n_1 k_1}{N_1}\right) \\
&= \sum_{n_1=0}^{N_1-1} \hat g_{n_2}[n_1] \exp \left (-j2\pi \frac{n_1 k_1}{N_1}\right),
\end{align}
$$

> la DFT 2D se puede calcular usando repetidas veces la DFT de una dimensión.

## Visualización de la base de Fourier bidimensional



In [ ]:
x = np.arange(0, 32, step=1)
X, Y = np.meshgrid(x, x)
fig, ax = plt.subplots(9, 9, figsize=(6, 6), tight_layout=True)

fourier_basis = np.cos
#fourier_basis = np.sin

for n in range(9):
    for m in range(9):
        ax[n, m].matshow(fourier_basis(2.0*np.pi*X*m/len(x) + 2.0*np.pi*Y*n/len(x)), 
                         cmap=plt.cm.RdBu_r, vmin=-1, vmax=1)
        ax[n, m].axis('off')

## Espectro de una imagen 

- Podemos usar la transformada de Fourier 2D para obtener el espectro de amplitud de una imagen
- Los ejes de la DFT son las frecuencias espaciales

In [ ]:
img_color = plt.imread('../images/valdivia.jpg')
img_bw = color2bw(img_color)[:, 300:]

fig, ax = plt.subplots(1, 2, figsize=(6, 3), tight_layout=True)
ax[0].imshow(img_bw, cmap=plt.cm.Greys_r) 
ax[0].set_title("Imagen")
S = fftpack.fft2(img_bw)
ax[1].imshow(fftpack.fftshift(np.abs(S)), cmap=plt.cm.Spectral_r,
             extent=[-img_bw.shape[1]//2, img_bw.shape[1]//2, 
                     -img_bw.shape[0]//2, img_bw.shape[0]//2])
ax[1].set_title("Espectro de amplitud");

La energía está muy concentrada en el componente central!

Para visualizar mejor el espectro de una imagen natural se recomienda usar

$$
\log(|\text{fft2}(I)|+1)
$$

de esta forma el componente central no es tan dominante

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 3), tight_layout=True)
ax[0].imshow(img_bw, cmap=plt.cm.Greys_r) 
ax[0].set_title("Imagen")
S = fftpack.fft2(img_bw)
ax[1].imshow(fftpack.fftshift(np.log(1+np.abs(S))), cmap=plt.cm.Spectral_r,
             extent=[-img_bw.shape[1]//2, img_bw.shape[1]//2, 
                     -img_bw.shape[0]//2, img_bw.shape[0]//2])
ax[1].set_title("Espectro de amplitud");

> ¿A qué corresponde las estructuras que aparecen en el espectro de amplitud? 

Para entrenarnos en la interpretación del espectro conviene estudiar espectros de imágenes sintéticas

## Espectro de una imagen sintética


#### Impulsos

Dos impulsos en el espectro corresponde a una sinusoide en el espacio original y viceversa

La posición de los impulsos está asociada a la frecuencia de la sinusoide

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(x_pos=0, y_pos=0):
    S_img = np.zeros(shape=(80, 80)); 
    S_img[x_pos, y_pos] = 1.0; S_img[-x_pos, -y_pos] = 1.0;
    #S_img[-x_pos, y_pos] = 1.0; S_img[x_pos, -y_pos] = 1.0; 
    ax[1].set_title("Espectro de amplitud");  
    ax[0].set_title("Imagen")
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r,
                      extent=[-40, 40, 40, -40])
    im = ax[0].imshow(np.real(fftpack.ifft2(S_img)), cmap=plt.cm.Greys_r);     
interact(update, 
         x_pos=IntSlider(min=-39, max=39, value=1, description="x posición"),
         y_pos=IntSlider(min=-39, max=39, value=0, description="y posición"));

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(x_f=0, y_f=0):
    x = np.arange(0, 80)
    X, Y = np.meshgrid(x, x)
    img = np.cos(2.0*np.pi*X*x_f/80 + 2.0*np.pi*Y*y_f/80)
    # img = np.cos(2.0*np.pi*X*x_f/80) + np.cos(2.0*np.pi*Y*y_f/80)
    # img = np.cos(2.0*np.pi*X*x_f/80)*np.cos(2.0*np.pi*Y*y_f/80)
    S_img = np.abs(fftpack.fft2(img))    
    ax[1].set_title("Espectro de amplitud");  ax[0].set_title("Imagen")
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r,
                      extent=[-40, 40, 40, -40])
    im = ax[0].imshow(img, cmap=plt.cm.Greys_r); 
    
interact(update, 
         x_f=IntSlider(min=0, max=39, value=0, description="x frecuencia"),
         y_f=IntSlider(min=0, max=39, value=0, description="y frecuencia"));

#### Espectro de una Linea

Una línea en la imagen es una línea en el espectro (con otra orientación) 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(pix_angle=0):
    img = np.zeros(shape=(80, 80));     
    for i in range(80):
        img[i, int(i - 2*pix_angle*i/80 + pix_angle)] = 1 
    ax[1].set_title("Magnitude spectrum");  
    ax[0].set_title("Image")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r, 
                      extent=[-40, 40, 40, -40])
    im = ax[0].imshow(img, cmap=plt.cm.Greys_r); 
    
interact(update, pix_angle=IntSlider(min=0, max=79, value=0));

##### ¿Cómo se explica esto? 

> Consideremos primero el caso donde la rotación es 40 pixeles (linea vertical)

Notemos que en este caso la componente vertical y horizontal pueden independizarse

Esto se conoce como **señal separable**

In [ ]:
A = np.array([[0, 0, 1, 0, 0]])
B = np.array([[1, 1, 1, 1, 1]])
A*B.T

#### Propiedad:  

> La DFT 2D de una señal separable equivale a la multiplicación de las dos DFT 1D

Una señal es sepable si

$$
g[n_1, n_2] = g_1[n_1] g_2[n_2]
$$

Aplicando esto en la DFT
$$
\begin{align}
G[k_1, k_2] &= \sum_{n_1=0}^{N_1-1} \sum_{n_2=0}^{N_2-1} g_1[n_1] g_2[n_2] \exp \left ( -j2\pi  \left[\frac{n_1 k_1}{N_1} + \frac{n_2 k_2}{N_2} \right] \right) \nonumber \\
& = \sum_{n_1=0}^{N_1-1} g_1[n_1] \exp \left ( -j2\pi \frac{n_1 k_1}{N_1} \right)  \sum_{n_2=0}^{N_2-1} g_2[n_2] \exp \left ( -j2\pi\frac{n_2 k_2}{N_2}  \right) \nonumber \\
\end{align}
$$

Usemos esta propiedad en nuestro ejemplo

Transformada de Fourier de un impulso en el origen: Una señal constante

In [ ]:
S1 = fftpack.fftshift(np.abs(fftpack.fft(np.array([[0, 0, 1, 0, 0]]))))
print(S1)

Transformada de Fourier de una señal constante: Un impulso en el origen

In [ ]:
S2 = fftpack.fftshift(np.abs(fftpack.fft(np.array([[1, 1, 1, 1, 1]]))))
print(S2)

**Resultado:** Una linea rotada en 90º con respecto a la original

In [ ]:
S1*S2.T

> Concentremos ahora en los ángulos (en píxeles) distintos de 0, 40 y 80 

¿Por qué pareciera repetirse la linea?

¿A qué corresponde el efecto  observado?

Para responder esta pregunta debemos recordar una propiedad de la DFT

#### Propiedad: 

>La DFT es periodica

En este caso el "artefacto" que observamos se debe a que en ciertos ángulos los bordes no calzan

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(6, 3.5), tight_layout=True); 

def update(pix_angle=0):
    img = np.zeros(shape=(60, 60));
    for i in range(120):
        for k1 in range(-4, 4):
            for k2 in range(-4, 4):
                if np.abs(i+20*k2) < 60 and np.abs(20*k1 + int(i - 2*pix_angle*i/20 + pix_angle)) < 60:
                    img[i +20*k2, 20*k1 + int(i - 2*pix_angle*i/20 + pix_angle)] = 1 
    
    ax[0].matshow(img, cmap=plt.cm.Greys_r); 
    ax[1].matshow(img[20:-20,20:-20], cmap=plt.cm.Greys_r); 
interact(update, pix_angle=IntSlider(min=0, max=20, value=0));

#### Espectro de un rectangulo

Un rectangulo en la imagen es un sinc en el espectro (y viceverza)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(width=1):
    img = np.zeros(shape=(80, 80)); #S_img[0, 0] = 0.0;
    img[40-width:40+width, 40-width:40+width] = 1.0; 
    ax[1].set_title("Espectro de amplitud")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r,
                      extent=[-40, 40, 40, -40])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img, cmap=plt.cm.Greys_r); 

interact(update, width=IntSlider(min=1, max=39, value=0, description="Tamaño"));

#### Espectro de una Gaussiana

Una Gaussiana en el espectro es una Gaussiana en el espacio original

In [ ]:
from ipywidgets import FloatSlider
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(bandwidth=1.0):
    x = np.linspace(-100, 100, num=500)
    X, Y = np.meshgrid(x, x)
    img = np.exp(-0.5*(X**2 + Y**2)/bandwidth**2)
    ax[1].set_title("Espectro de amplitud")
    S_img = np.abs(fftpack.fft2(img))
    im = ax[1].imshow(fftpack.fftshift(S_img),  cmap=plt.cm.Spectral_r,
                      extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img, cmap=plt.cm.Greys_r);     
    
interact(update, bandwidth=FloatSlider(min=0.1, max=50.0, value=0, description="Ancho"));

#### Espectro de ruido blanco

Una imagen de ruido blanco y su espectro

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def update(rseed=1.0):
    np.random.seed(rseed)
    img = np.random.randn(500, 500) 
    ax[1].set_title("Espectro de amplitud")
    S_img = np.log10(1e-6 + np.abs(fftpack.fft2(img)))
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r,
                      extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(img, cmap=plt.cm.Greys_r); 

interact(update, rseed=IntSlider(min=0, max=100, value=0, description="rseed"));

#### Espectro de ruido rojo

Una imagen de ruido rojo y su espectro

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True); 

def f(rseed=1.0, gamma=0.0):
    np.random.seed(rseed)
    red_noise = np.random.randn(500, 500) 
    rho = 1-10**-gamma
    #for i in range(2,500):
    #    red_noise[i, :] = rho*red_noise[i-1, :] + red_noise[i, :]
    #    red_noise[:, i] = rho*red_noise[:, i-1] + red_noise[:, i]
    for i in range(2, 500):
        for j in range(2, 500):
            red_noise[i, j] += rho*np.average(red_noise[i-2:i, j-2:j])
    ax[1].set_title("Espectro de amplitud")
    S_img = np.log10(1e-10 + np.abs(fftpack.fft2(red_noise)))
    im = ax[1].imshow(fftpack.fftshift(S_img), cmap=plt.cm.Spectral_r,
                      extent=[-250, 250, 250, -250])
    ax[0].set_title("Imagen")
    im = ax[0].imshow(red_noise, cmap=plt.cm.Greys_r); 

interact(f, 
         rseed=IntSlider(min=0, max=100, value=0, description="rseed"),
         gamma=FloatSlider(min=0, max=3, value=0, description="$\gamma$"));

## Espectro de una imagen natural

Ahora que hemos aprendido a interpretar el los espectros, analizemos una imagen natural

In [ ]:
img_building = color2bw(plt.imread('../images/building.jpg'))
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)

im = ax[0].imshow(img_building, cmap=plt.cm.Greys_r)
S_img = fftpack.fft2(img_building)
im = ax[1].imshow(fftpack.fftshift(np.log(np.abs(S_img)+1)), cmap=plt.cm.Spectral_r)

> Las fuerte componentes horizontal y vertical son en realidad un artefacto de borde 

La DFT considera que la señal 2D es eternamente periódica 

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), tight_layout=True)
ax.imshow(np.tile(img_building, (3, 3)), cmap=plt.cm.Greys_r)
ax.axis('off');

Puedes disminuir este efecto usando **enventanado**

In [ ]:
win = np.hanning(img_building.shape[0]).reshape(-1, 1)
win = np.dot(win, win.T)

fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)
im = ax[0].imshow(img_building, cmap=plt.cm.Greys_r)
S_img = fftpack.fft2(win*img_building)
im = ax[1].imshow(fftpack.fftshift(np.log(np.abs(S_img)+1)), cmap=plt.cm.Spectral_r)

Finalmente:  

¿Puedes reconocer a que corresponden las estructuras en el espectro del siguiente ejemplo?

## Espectro de magnitud y de fase

La DFT 2D es un número complejo

Hasta ahora hemos estudiado su valor absoluto

>  La **magnitud espectral** guarda información de la amplitud de los componentes

Ahora veremos que influencia tiene la

> La **fase espectral** guarda información del desfase (posición) de los componentes

In [ ]:
img_doge = color2bw(plt.imread('../images/doge.jpg')) 

plt.figure(figsize=(6, 4), tight_layout=True)
plt.imshow(img_doge, cmap=plt.cm.Greys_r)

¿Cómo se ven los espectros de magnitud y fase de esta imagen?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4), tight_layout=True)
S_img = fftpack.fft2(img_doge)
im = ax[0].imshow(fftpack.fftshift(np.log(1.+np.abs(S_img))), cmap=plt.cm.Spectral_r)
fig.colorbar(im, ax=ax[0], orientation='horizontal')
im = ax[1].imshow(fftpack.fftshift(np.angle(S_img)), cmap=plt.cm.Spectral_r)  
fig.colorbar(im, ax=ax[1], orientation='horizontal');

> ¿Podemos reconstruir usando sólo el espectro de magnitud? ¿O usando sólo el de fase?

In [ ]:
def hist_eq(img, nbins=256):
    image_hist, bins = np.histogram(img.flatten(), nbins, density=True)
    cdf = image_hist.cumsum() 
    cdf = 255 * cdf / cdf[-1] 
    image_eq = np.interp(img.flatten(), bins[:-1], cdf)
    return image_eq.reshape(img.shape).astype('int')
    #return img

fig, ax = plt.subplots(2, 2, figsize=(8, 6), tight_layout=True)
for ax_ in ax.ravel():
    ax_.axis('off')

ax[0, 0].imshow(img_doge, cmap=plt.cm.Greys_r);
S_dog = fftpack.fft2(img_doge)
ax[0, 1].hist(img_doge.ravel(), alpha=0.5, bins=100); 
ax[0, 1].hist(hist_eq(img_doge.ravel()), alpha=0.5, bins=100);ax[0, 1].axis('on')
reconstruct = lambda S: np.real(fftpack.ifft2(S))
ax[1, 0].imshow(hist_eq(reconstruct(np.abs(S_dog))), cmap=plt.cm.Greys_r);
ax[1, 1].imshow(hist_eq(reconstruct(np.exp(1j*np.angle(S_dog, deg=False)))), cmap=plt.cm.Greys_r);

¿Y si intercambiamos la fase y magnitud de dos imágenes de igual tamaño?

In [ ]:
img_inst = color2bw(plt.imread("../images/InsInformatica.jpg"))  
fig, ax = plt.subplots(2, 2, figsize=(8, 5), tight_layout=True)
for ax_ in ax.ravel():
    ax_.axis('off')

ax[0, 0].imshow(img_doge, cmap=plt.cm.Greys_r);
ax[0, 1].imshow(img_inst, cmap=plt.cm.Greys_r); 
S_inf = fftpack.fft2(img_inst)
rec_doge = fftpack.ifft2(np.abs(S_dog)*np.exp(1j*np.angle(S_inf, deg=False)))
rec_inst = fftpack.ifft2(np.abs(S_inf)*np.exp(1j*np.angle(S_dog, deg=False)))
ax[1, 0].set_title('Amplitud doge\nAngulo instituto')
ax[1, 0].imshow(np.real(rec_doge), cmap=plt.cm.Greys_r); 
ax[1, 1].set_title('Amplitud instituto\nAngulo doge')
ax[1, 1].imshow(np.real(rec_inst), cmap=plt.cm.Greys_r); 